In [1]:
####### 기본 블록
from torch import nn


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1):
        # 기본 블록 합성곱 계층의 차원 64, 128, 256, 512
        # 입ㅕㄱ 차원 수와 출력 차원 수 매개변수 필요 (inplanes, planes)
        super().__init__()
        self.conv1 = nn.Conv2d(
            inplanes, planes,
            kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            planes, planes,
            kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        # 입력값과 출력값의 차원이 다른 경우
        if stride != 1 or inplanes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    # 잔차연결을 위해 입력값과 출력값 차원 맞추기
                    # inplanes -> expension*planes
                    inplanes, self.expansion*planes,
                    kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        # 3x3 특징 맵 2계층
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        # 잔차 연결 (+=)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [2]:
####### 병목 블록
class BottleneckBlock(nn.Module):
    # resnet-50, 101, 152 경우 모든 스테이지에서 출력차원 = 4x입력차원
    expansion = 4

    def __init__(self, inplanes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(
            inplanes, planes,
            kernel_size=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, planes,
            kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(
            planes, self.expansion*planes,
            kernel_size=1, bias=False
        )
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)
        self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or inplanes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    inplanes, self.expansion*planes,
                    kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [3]:
####### 레즈넷 모델 클래스
import torch


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super().__init__()

        self.inplanes = 64
        #### input stem
        self.stem = nn.Sequential(
            # 입력은 이미지이므로 3차원, 출력은 64차원
            nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.inplanes),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        ) 
        #### stage
        self.stage1 = self._make_layer(block, 64, layers[0], stride=1)
        # 두번째 스테이지부터 첫번째 블록의 첫번째 합성곱 stride=2 적용
        # 병목블록에선 첫번째 블록의 두번째 합성곱 stride=2 적용
        # VGG의 최댓값 풀링을 대신한 stride를 통해 특징 맵 크기 감소
        self.stage2 = self._make_layer(block, 128, layers[1], stride=2)
        self.stage3 = self._make_layer(block, 256, layers[2], stride=2)
        self.stage4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        layers = []
        # 두번째 스테이지부터 첫번째 블록의 첫번째 합성곱 stride=2 적용
        # 병목블록에선 첫번째 블록의 두번째 합성곱 stride=2 적용
        layers.append(block(self.inplanes, planes, stride))
        self.inplanes = planes * block.expansion
        for _ in range(num_blocks - 1):
            layers.append(block(self.inplanes, planes, 1))
        
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.stem(x)
        out = self.stage1(out)
        # 두 번째 스테이지~마지막 스테이지의 첫 번째 블록의 첫 번째 합성곱엔 stride = 2
        # 병목블록에선 첫번째 블록의 두번째 합성곱 stride=2 적용
        out = self.stage2(out)
        out = self.stage3(out)
        out = self.stage4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

In [4]:
####### 레즈넷 모델 비교
from torchvision import models
from torchinfo import summary


resnet18    = ResNet(BasicBlock, [2, 2, 2, 2], 1000)
resnet34    = ResNet(BasicBlock, [3, 4, 6, 3], 1000)
resnet50    = ResNet(BottleneckBlock, [3, 4, 6, 3], 1000)
resnet101   = ResNet(BottleneckBlock, [3, 4, 23, 3], 1000)
resnet152   = ResNet(BottleneckBlock, [3, 8, 36, 3], 1000)
torch_model = models.resnet34(weights="ResNet34_Weights.IMAGENET1K_V1")

resnet34_info    = summary(resnet34, (1, 3, 224, 224), verbose=0)
torch_model_info = summary(torch_model, (1, 3, 224, 224), verbose=0)

print(resnet34_info.total_params)
print(torch_model_info.total_params)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /Users/seoyun/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 78.8MB/s]


21797672
21797672
